<a href="https://colab.research.google.com/github/PingMoon93/PingMoon93/blob/main/IS41070_Machine_Learning_Foundations_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TASK 1 DATA UNDERSTANDING

1.Download the dataset.

2.Upload the dataset.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/IS41070-MACHINE LEARNING FOUNDATIONS/92.csv')

3.Exploration of the data.

In [ ]:
data.head()

In [ ]:
#Get a quick outview of the dataset.
data.info()

In [ ]:
#Get the exact number of each category
data['category'].unique()
categories = data['category'].unique()
caterories_count = data['category'].value_counts()
print(caterories_count)

In [ ]:
#check for the missing value
missing_value = data.isnull().sum()
missing_value

In [ ]:
#Generate the wordcloud to have a visualization of the two categories
from wordcloud import WordCloud
science_text =' '.join(data[data['category'] == 'SCIENCE' ]['short_description'].fillna('').astype(str))
wordcloud = WordCloud(width=800, height=800, background_color='black').generate(science_text)
plt.figure(figsize=(8, 8))
plt.imshow(wordcloud)
plt.axis('off')
plt.title(f'Word Cloud for SCIENCE description')
plt.show()

In [ ]:
wellness_text =' '.join(data[data['category'] == 'WELLNESS' ]['short_description'].fillna('').astype(str))
wordcloud = WordCloud(width=800, height=800, background_color='black').generate(wellness_text)
plt.figure(figsize=(8, 8))
plt.imshow(wordcloud)
plt.axis('off')
plt.title(f'Word Cloud for WELLNESS description')
plt.show()

In [ ]:
#word frequency analysis
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from collections import Counter
import matplotlib.pyplot as plt
import re

In [ ]:
#For brevity, we'll wrap preprocessing and term frequency analysis into functions
#Remove non-word, delete single standalone
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text)  #
    return text.strip()
#tokenize and stop words, count frequency
def calculate_word_frequencies(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    word_freq = Counter(filtered_tokens)
    return word_freq
#print most 10 common words
def plot_word_frequencies(word_freq):
    common_words = word_freq.most_common(10)
    words, counts = zip(*common_words)
    plt.figure(figsize=(10, 5))
    plt.bar(words, counts)
    plt.xlabel('Words')
    plt.ylabel('Frequencies')
    plt.title('Top 10 Word Frequencies')
    plt.show()

In [ ]:
category = 'WELLNESS'
wellness_descriptions = data[data['category'] == category]['short_description'].fillna('').astype(str)
#join the text
wellness_text = ' '.join(wellness_descriptions)
# preprocess the text and calculate the frequency
cleaned_text = preprocess_text(wellness_text)
word_freq = calculate_word_frequencies(cleaned_text)
plot_word_frequencies(word_freq)

In [ ]:
category = 'SCIENCE'
science_descriptions = data[data['category'] == category]['short_description'].fillna('').astype(str)
#join the text
wellness_text = ' '.join(science_descriptions)

# preprocess the text and calculate the frequency
cleaned_text = preprocess_text(science_text)
word_freq = calculate_word_frequencies(cleaned_text)
plot_word_frequencies(word_freq)

In [ ]:
#Calculate statistics about the length of different categories of short description text
text_lengths_wellness = data[data['category'] == 'WELLNESS']['short_description'].apply(lambda x: len(str(x)))
text_lengths_wellness_mean = text_lengths_wellness.mean()
text_lengths_wellness_median = text_lengths_wellness.median()
text_lengths_science = data[data['category'] == 'SCIENCE']['short_description'].apply(lambda x: len(str(x)))
text_lengths_science_mean = text_lengths_science.mean()
text_lengths_science_median = text_lengths_science.median()
print(f"The mean length of short descriptions in the WELLNESS category is: {text_lengths_wellness_mean}")
print(f"The median length of short descriptions in the WELLNESS category is: {text_lengths_wellness_median}")
print(f"The mean length of short descriptions in the SCIENCE category is: {text_lengths_science_mean}")
print(f"The median length of short descriptions in the SCIENCE category is: {text_lengths_science_median}")


In [ ]:
#Calculate statistics about the length of different categories of headline
headline_lengths_wellness = data[data['category'] == 'WELLNESS']['headline'].apply(lambda x: len(str(x)))
headline_lengths_wellness_mean = headline_lengths_wellness.mean()
headline_lengths_wellness_median = headline_lengths_wellness.median()
headline_lengths_science = data[data['category'] == 'SCIENCE']['headline'].apply(lambda x: len(str(x)))
headline_lengths_science_mean = headline_lengths_science.mean()
headline_lengths_science_median = headline_lengths_science.median()
print(f"The mean length of headline in the WELLNESS category is: {headline_lengths_wellness_mean}")
print(f"The median length of headline in the WELLNESS category is: {headline_lengths_wellness_median}")
print(f"The mean length of headline in the SCIENCE category is: {headline_lengths_science_mean}")
print(f"The median length of headline in the SCIENCE category is: {headline_lengths_science_median}")


We can see that there are 5984 articles in the wellness category, 1,996 articles in the science category, and 20 articles with no category information.In addition, categories are unevenly distributed, with wellness samples much higher than science, and subsequent training of the model needs to focus on this.

The word cloud and word frequencies of the short descriptions reflect the characteristics of the two categories to a certain extent. The high-frequency words of the health category include life, people, health and so on, which are clearly related to health. The high-frequency words scientists, space, earth, etc. of the science category are also obviously related to scientific topics.

As for the length of the sentences, there is a clear difference across categories, clearly the average sentence length of wellness short description is significantly longer than that of science category, as is the median.As for the length of the title, it is the fact that the average title length and median of the Science category are higher than those of the wellness category.


TASK2  Data Preparation & Modelling

4.Splitting the dataset

In [ ]:
#This task focuses on building a model for classifying text
#so we first remove the data with missing categories
data_cleaned = data.dropna(subset=['category'])
data_cleaned.info()

In [ ]:
from sklearn.model_selection import train_test_split
#The dataset is divided into a training set and a temporary set at a ratio of 6:4
#And then the temporary set is divided into a test set and a validation set at a ratio of 1:1
train_data,temp_data = train_test_split(data_cleaned,test_size=0.4, random_state=42)
test_data, valid_data = train_test_split(temp_data,test_size=0.5,random_state=42)

In [ ]:
#The split datasets were saved separately as csv files and saved in google drive
train_file_path = '/content/drive/MyDrive/Machine Learning/IS41070-MACHINE LEARNING FOUNDATIONS/train data.csv'
test_file_path = '/content/drive/MyDrive/Machine Learning/IS41070-MACHINE LEARNING FOUNDATIONS/test data.csv'
valid_file_path = '/content/drive/MyDrive/Machine Learning/IS41070-MACHINE LEARNING FOUNDATIONS/valid data.csv'

In [ ]:
#Save the data to csv
train_data.to_csv(train_file_path, index=False)
test_data.to_csv(test_file_path, index=False)
valid_data.to_csv(valid_file_path, index=False)

5.Load the train and valid data and apply preprocess steps.

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Machine Learning/IS41070-MACHINE LEARNING FOUNDATIONS/train data.csv')
valid_data = pd.read_csv('/content/drive/MyDrive/Machine Learning/IS41070-MACHINE LEARNING FOUNDATIONS/valid data.csv')

In [ ]:
#Remove irrelevant information
train_data = train_data.drop(columns=['link','authors','date'])
valid_data = valid_data.drop(columns=['link','authors','date'])

In [ ]:
train_data.info()

In [ ]:
valid_data.info()

In [ ]:
import re
import string
# define preprocess function and name it as clean_text
def clean_text(text):
    '''
    Perform stop-words removal and lemmatization
    '''
    text = text.lower()
    text = re.sub(r"[^a-zA-Z?.!,¿]+|http\S+", " ", text)
    text = ''.join([char for char in text if char not in string.punctuation])
    words = [word for word in text.split() if word not in stopwords.words('english')]
    words = [WordNetLemmatizer().lemmatize(word) for word in words]
    return " ".join(words)

5.2Compare whether there are differences in modeling before and after preprocessing

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from nltk.stem import WordNetLemmatizer

In [ ]:
#Use the fillna () function to fill the missing values with empty strings
valid_data['short_description'].fillna('', inplace=True)
train_data['short_description'].fillna('', inplace=True)

In [ ]:
#Unpreprocess
x_train1 = train_data['short_description'].astype(str)
y_train1 = train_data['category']
x_valid1 = valid_data['short_description'].astype(str)
y_valid1 = valid_data['category']
vectorizer = TfidfVectorizer(max_features=5000)
x_train_tf1 = vectorizer.fit_transform(x_train1)
x_valid_tf1 = vectorizer.transform(x_valid1)

In [ ]:
model1 = LogisticRegression()

In [ ]:
#Train the model
model1.fit(x_train_tf1,y_train1)

In [ ]:
y_pred1 = model1.predict(x_valid_tf1)

In [ ]:
print(classification_report(y_valid1,y_pred1))

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
#With preprocess steps
train_data['short_description_pre'] = train_data['short_description'].apply(clean_text)
valid_data['short_description_pre'] = valid_data['short_description'].apply(clean_text)

In [ ]:
train_data.head()

In [ ]:
x_train2 = train_data['short_description_pre'].astype(str)
y_train2 = train_data['category']
x_valid2 = valid_data['short_description_pre'].astype(str)
y_valid2 = valid_data['category']
vectorizer = TfidfVectorizer(max_features=5000)
x_train_tf2 = vectorizer.fit_transform(x_train2)
x_valid_tf2 = vectorizer.transform(x_valid2)

In [ ]:
model2 = LogisticRegression()
model2.fit(x_train_tf2,y_train2)

In [ ]:
y_pred2 = model2.predict(x_valid_tf2)
print(classification_report(y_valid2,y_pred2))

Based on the above analysis, Model1 has better overall performance in the two categories, especially in the recall rate and F1-score of the SCIENCE category, which can be considered as a better model choice. Without text processing, Model 1 draws conclusions that contradict our general belief that text preprocessing is beneficial for model tuning.

6.Build binary classification models

Having trained the logistic regression model in Task 5, let's use the same method to train the Random Forest model.

In [ ]:
#Build RandomForrest model with unpreprocess text
from sklearn.ensemble import RandomForestClassifier
rf_model1 = RandomForestClassifier()

In [ ]:
#Train the RandomForrest model
rf_model1.fit(x_train_tf1,y_train1)

In [ ]:
y_pred1_rf = rf_model1.predict(x_valid_tf1)
print(classification_report(y_valid1,y_pred1_rf))

In [ ]:
#Build another RandomForrest Model using preprocess text
rf_model2 = RandomForestClassifier()
rf_model2.fit(x_train_tf2,y_train2)

In [ ]:
y_pred2_rf = rf_model2.predict(x_valid_tf2)
print(classification_report(y_valid2,y_pred2_rf))

Integrating the evaluation metrics of the two models, rf_model2 performs better in the recall and overall macro average metrics of the SCIENCE category, while it is slightly better than rf_model1 in the precision and F1-score of the WELLNESS category. Therefore, it can be argued that rf_model2 is the better choice of the two models, especially when dealing with the SCIENCE samples. rf_model2 uses the preprocessed text for training, indicating that text preprocessing has a positive effect on the tuning of the random forest model.

7. Build an end-to-end classifier using deep learning

In [ ]:
#Using PyTorch and train the new model from scratch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re

In [ ]:
news_data = pd.read_csv('/content/drive/MyDrive/Machine Learning/IS41070-MACHINE LEARNING FOUNDATIONS/92.csv')
#drop the missing value
news_data.dropna(subset=['short_description','category'],inplace=True)

In [ ]:
news_data.info()

In [ ]:
#Text preprocess
news_data['short_description_pre'] = news_data['short_description'].apply(clean_text)
#Encoder
label_encoder = LabelEncoder()
news_data['category'] = label_encoder.fit_transform(news_data['category'])
#Split the data
x_train, x_test, y_train, y_test = train_test_split(news_data['short_description_pre'], news_data['category'], test_size=0.2, random_state=42)
#Convert text data to TF-IDF feature vectors
vectorizer = TfidfVectorizer(max_features=5000)
x_train_tfidf = vectorizer.fit_transform(x_train).toarray()
x_test_tfidf = vectorizer.transform(x_test).toarray()

In [ ]:
#Constructor to initialize the dataset object
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = torch.tensor(texts, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

In [ ]:
#Converts pandas objects to list
y_train = y_train.tolist()
y_test = y_test.tolist()
train_dataset = TextDataset(x_train_tfidf, y_train)
test_dataset = TextDataset(x_test_tfidf, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Define the module
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Set model parameters
input_dim = 5000  # Number of features after TF-IDF vectorization
hidden_dim = 100
output_dim = len(label_encoder.classes_)

# Build the model
model = TextClassifier(input_dim, hidden_dim, output_dim)

In [ ]:
# Define criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for texts, labels in train_loader:
        outputs = model(texts)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for texts, labels in test_loader:
        outputs = model(texts)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate accuracy
correct = sum(p == l for p, l in zip(all_predictions, all_labels))
total = len(all_labels)
accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy:.2f}%')

# Print classification report
print(classification_report(all_labels, all_predictions))

The above results show that the accuracy of the model on the test set is 86.44%, which means that 86.44% of the samples in the test data set are correctly classified by the model.

TASK3 Evaluation

8.Choose a primary metric to evaluate the models

We train five models: two logistic regression models (model1 and model2), where model1 is not text preprocessed and model2 is text preprocessed; There are also two random forest models (rf_model1 and rf_model2), where rf_model1 has no text preprocessing and rf_model2 has text preprocessing. The last model is a deep learning model trained using PyTorch.

We used classification reports to evaluate model performance. We mainly considered F1-score and accuracy as evaluation metrics, because accuracy intuitively reflects the overall prediction accuracy of the model, and F1-score combines precision and recall. More reliable when dealing with class imbalance (Kampakis, 2023).


9.Evaluate the performance of each model.

1.Comparing Model1 with Model2, which uses short description after text preprocessing for model training, Model1 performs significantly better than Model2 in the SCIENCE category, especially in the recall rate and F1-score. The recall rate of Model2 in the SCIENCE category is only 0.33, which may lead to missing many samples in the SCIENCE category. There is little difference between the precision of Model1 and Model2 in the WELLNESS category, but Model2 has a slightly higher recall, Model1 has a slightly higher F1-score in the WELLNESS category, and the overall performance is slightly better than Model2. In the overall evaluation, the weighted average metrics (precision, recall, and F1-score) of Model1 are better than those of Model2, indicating its overall better performance.Without text processing, Model 1 has a better performance, which draws conclusions that contradict our general belief that text preprocessing is beneficial for model training.

2.Comparing rf_model1 and rf_model2, the latter uses the short description after text preprocessing for model training, the precision of rf_model1 is slightly higher than that of rf_model2 in the SCIENCE category, but the recall rate is lower. As a result, the F1-score is also lower. The recall and F1-score of rf_model2 in the SCIENCE category are slightly better than those of rf_model1. Besides, the precision and F1-score of rf_model2 in the WELLNESS category are slightly higher than those of rf_model1. In particular, it has an advantage in recall. Integrating the evaluation metrics of the two models, rf_model2 performs better in the recall and overall macro average metrics of the SCIENCE category, while it is slightly better than rf_model1 in the precision and F1-score of the WELLNESS category. Therefore, it can be argued that rf_model2 is the better choice of the two models, especially when dealing with the SCIENCE samples.

3.Comparing model2 with rf_model2, the former is a logistic regression model, and the latter is a random forest model. In the SCIENCE category, the Precision of Model2 is 0.90, the Recall is 0.55, and the F1-score is 0.68. rf_model2's Precision is 0.94, Recall is 0.47, and F1-score is 0.63. Although rf_model2 is slightly higher in precision, Model2 performs better in recall and F1-score. This indicates that Model2 is more balanced and comprehensive in identifying the SCIENCE category. For the wellness category, the Precision of Model2 is 0.87, the Recall is 0.98, the F1-score is 0.92, the Precision of rf_model2 is 0.85, the Recall is 0.99, With an F1-score of 0.91, both models perform similarly on this category, but slightly higher on the Model2 metric. From the analysis of comprehensive performance indicators, Model2 is more balanced and comprehensive in processing data, especially in the case of class imbalance, its performance in each category is relatively better,which draws the conclusion that in this analysis, category 'headline' seems to be a better feature than 'short_description'.

In conclusion, among the four models trained, rf_model2 with short description after preprocessing has the best comprehensive performance.

10.Error analysis



Comparing model classification reports, except model 2, the accuracy of our trained models reached more than 85%. Through the analysis of classification reports, we found that the performance of WELLNESS and SCIENCE was different, and the precision and recall of all models in Wellness category were higher than those in SCIENCE category. In addition, The recall rate of the SCIENCE category is not high, and the reason for the above phenomenon is mainly the problem of class imbalance in the data set. The data of WELLNESS is 5984, and the data of SCIENCE is 1996, the former is almost 3 times that of the latter. In addition to this reason, the choice of feature value is too single is another reason. The model is trained with a single feature value, and can be trained with multiple feature values later.

Let's take a look at the part of the random forest and logistic regression model that predict wrong together, here we choose model2 and rf_model2 as a comparison:

In [ ]:
incorrect_indices_model2 = set(np.where(y_valid2 != y_pred2)[0])
incorrect_indices_rf_model2 = set(np.where(y_valid2 != y_pred2_rf)[0])

common_incorrect_indices = incorrect_indices_model2.intersection(incorrect_indices_rf_model2)
for index in common_incorrect_indices:
    print(f"Index: {index}, True Label: {y_valid2.iloc[index]}, Model 2 Predicted Label: {y_pred2[index]}, rf_model2 Predicted Label: {y_pred2_rf[index]}")
    print(f"Text: {x_valid2.iloc[index]}")
    print('-' * 80)

In [ ]:
print(f"Number of common incorrect predictions: {len(common_incorrect_indices)}")

In [ ]:
#Count the number of classification errors
error_counts = {}

for index in common_incorrect_indices:
    true_label = y_valid2.iloc[index]
    pred_label2 = y_pred2[index]
    pred_label3 = y_pred2_rf[index]

    # Error in model 2
    if true_label != pred_label2:
        if (true_label, pred_label2) not in error_counts:
            error_counts[(true_label, pred_label2)] = 0
        error_counts[(true_label, pred_label2)] += 1

    # Error in rf_model2
    if true_label != pred_label3:
        if (true_label, pred_label3) not in error_counts:
            error_counts[(true_label, pred_label3)] = 0
        error_counts[(true_label, pred_label3)] += 1

# Print the number of classification errors
print("Error counts:")
for (true_label, pred_label), count in error_counts.items():
    print(f"True Label: {true_label}, Predicted Label: {pred_label}, Count: {count}")


The above results show that both models make a total of 129 errors, and the number of misclassified SCIENCE as WELLNESS is 242, far more than the number of misclassified WELLNESS as SCIENCE is 16, indicating that when the model distinguishes between WELLNESS and SCIENCE, The better recognition ability for the WELLNESS category may be related to the fact that there are more samples in the WELLNESS category in the sample.

By analyzing the parts of the text that were incorrectly Predicted by both models, we found that the misprediction may also result from the presence of high-frequency words in another category in these texts, for example, "Index: 600, True Label: SCIENCE, Model 2 Predicted Label: WELLNESS, rf_model2 Predicted Label: WELLNESS
Text:  one important part pet owner looking pet health well ",both models predicted the original SCIENCE to be WELLNESS. health appears in this text, which is a high-frequency word in the WELLNESS category.

From this analysis, I learned how to find the samples where the two models jointly predicted wrong, and analyze these samples. By looking at the wrong samples, I can identify the possible classification problems of the model, such as the high-frequency vocabulary problem and the data imbalance problem mentioned above. Understanding the possible reasons for the error can help improve the model. Different feature values, such as headline, can be selected to improve the model in subsequent research.

11.Model optimization

We have already analyzed the influence of preprocess, let's use headline as the factor to train the model.

In [ ]:
#Use headline as factor and build a new logic model.
#Use the fillna() function to fill the missing values.
train_data['headline'].fillna('',inplace=True)
valid_data['headline'].fillna('',inplace=True)

In [ ]:
#Preprocess the headline
train_data['headline_pre']=train_data['headline'].apply(clean_text)
valid_data['headline_pre']=valid_data['headline'].apply(clean_text)

In [ ]:
train_data.head()

In [ ]:
x_train3= train_data['headline_pre'].astype(str)
y_train3= train_data['category']
x_valid3= valid_data['headline_pre'].astype(str)
y_valid3= valid_data['category']
vectorizer = TfidfVectorizer(max_features=5000)
x_train_tf3 = vectorizer.fit_transform(x_train3)
x_valid_tf3 = vectorizer.transform(x_valid3)
model3 = LogisticRegression()
model3.fit(x_train_tf3,y_train3)

In [ ]:
y_pred3 = model3.predict(x_valid_tf3)
print(classification_report(y_valid3,y_pred3))


Comparing Model 2 and Model 3, Model 3 performs slightly better than Model 2, especially in the precision and recall of the SCIENCE category, and for the WELLNESS category the two models are very close, and model 3 has a slight advantage in recall. In terms of dealing with class imbalance, Model 3 is better than Model 2 in all indicators, indicating that choosing headline as feature value and using logistic regression model is better than choosing short description.

In [ ]:
#Use headline as factor to train a Random Forrest model
rf_model3 = RandomForestClassifier()
rf_model3.fit(x_train_tf3,y_train3)

In [ ]:
y_pred3_rf = rf_model3.predict(x_valid_tf3)
print(classification_report(y_valid3,y_pred3_rf))



Comparing rf_model2 and rf_model3, the former uses short description, and the latter uses headline as feature value, it is found that rf_model3 has improved. It outperforms rf_model2 on most metrics, especially in handling class imbalance and improving recall for the SCIENCE category.

According to the above conclusions, the headline is selected as the feature value, and the model ability is optimized, and the accuracy of the optimized model has reached to 90%. Next, let's optimize the deep learning model.

In [ ]:
news_data1 = pd.read_csv('/content/drive/MyDrive/Machine Learning/IS41070-MACHINE LEARNING FOUNDATIONS/92.csv')

In [ ]:
news_data1.dropna(subset=['headline','category'],inplace=True)

In [ ]:
news_data1.info()

In [ ]:
#Text preprocess
news_data1['headline_pre'] = news_data1['headline'].apply(clean_text)
#Encoder
label_encoder = LabelEncoder()
news_data1['category'] = label_encoder.fit_transform(news_data1['category'])
#Split the data
x_train1, x_test1, y_train1, y_test1 = train_test_split(news_data1['headline_pre'], news_data1['category'], test_size=0.2, random_state=42)
#Convert text data to TF-IDF feature vectors
vectorizer = TfidfVectorizer(max_features=5000)
x_train_tfidf1 = vectorizer.fit_transform(x_train1).toarray()
x_test_tfidf1 = vectorizer.transform(x_test1).toarray()

In [ ]:
news_data1.info()

In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.3)  # Change Dropout layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout after activation
        x = self.fc2(x)
        return x

# Assuming you have defined train_loader and test_loader properly

# Set model parameters
input_dim = 5000  # Number of features after TF-IDF vectorization
hidden_dim = 100
output_dim = len(label_encoder.classes_)

# Build the model
model = TextClassifier(input_dim, hidden_dim, output_dim)

# Define criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5  # Reduce epochs to 5
for epoch in range(num_epochs):
    model.train()
    for texts, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate on test set
model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for texts, labels in test_loader:
        outputs = model(texts)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate accuracy
correct = sum(p == l for p, l in zip(all_predictions, all_labels))
total = len(all_labels)
accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy:.2f}%')

# Print classification report
print(classification_report(all_labels, all_predictions))


Choosing headline as the feature value and decreasing epoch to 5 and increasing dropput to 0.3 show a slight improvement in model performance. Compared with the first trained deep learning model, the Accuracy of the optimized model (0.87 vs 0.86) is improved.
The Precision, Recall and F1-Score of the optimized models Macro Avg and Weighted Avg are better than those before optimization, showing better balance and comprehensive performance.

12.Merge train_data and valid_data, and perform cross validation.

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Machine Learning/IS41070-MACHINE LEARNING FOUNDATIONS/train data.csv')
valid_data = pd.read_csv('/content/drive/MyDrive/Machine Learning/IS41070-MACHINE LEARNING FOUNDATIONS/valid data.csv')

In [ ]:
#merge the train data and valid data and name it as combined data
combined_data = pd.concat([train_data,valid_data], ignore_index=True)

In [ ]:
combined_data.info()

In [ ]:
combined_data.dropna(subset=['headline','category'],inplace=True)

In [ ]:
from sklearn.model_selection import cross_val_score,KFold
from sklearn.linear_model import LogisticRegression
x = combined_data['headline'].astype(str)
y = combined_data['category']
vectorizer = TfidfVectorizer(max_features=5000)
x_tfidf = vectorizer.fit_transform(x).toarray()
model3 = LogisticRegression()
rf_model3 = RandomForestClassifier()

In [ ]:
# Define the number of folds for cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and evaluate model performance
cv_scores_model3 = cross_val_score(model3, x_tfidf, y, cv=kf, scoring='accuracy')
cv_scores_rf_model3 = cross_val_score(rf_model3, x_tfidf, y, cv=kf, scoring='accuracy')

# Print cross-validation results
print("Logistic Regression Cross-validation scores:", cv_scores_model3)
print("Logistic Regression Mean CV accuracy:", cv_scores_model3.mean())

print("Random Forest Cross-validation scores:", cv_scores_rf_model3)
print("Random Forest Mean CV accuracy:", cv_scores_rf_model3.mean())


The above results show that both models exhibit relatively high cross-validation accuracy, indicating that they can learn and make accurate predictions on the dataset, where the random forest model has a higher average accuracy than the logistic regression model, indicating that the trained random forest model is slightly better than the logistic regression model.

In [ ]:
#Perform cross validation of PyTorch model
#(How to Use K-fold Cross Validation With DataLoaders in PyTorch | Saturn Cloud Blog, 2024)
# Convert text data to TF-IDF features
from sklearn.metrics import accuracy_score
vectorizer = TfidfVectorizer(max_features=5000)
x_tfidf = vectorizer.fit_transform(x).toarray()

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Define model hyperparameters
input_dim = 5000
hidden_dim = 100
output_dim = len(label_encoder.classes_)

# Define KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Prepare to store cross-validation results
cv_scores = []

# Define training and evaluation function
def train_and_evaluate(train_index, val_index):
    # Create the model
    model = TextClassifier(input_dim, hidden_dim, output_dim)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Get training and validation data
    x_train, x_val = x_tfidf[train_index], x_tfidf[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Convert to tensors
    x_train = torch.tensor(x_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    x_val = torch.tensor(x_val, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.long)

    # Train the model
    num_epochs = 5
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(x_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

    # Validate the model
    model.eval()
    with torch.no_grad():
        outputs = model(x_val)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_val.numpy(), predicted.numpy())

    return accuracy

# Perform cross-validation
for train_index, val_index in kf.split(x_tfidf):
    accuracy = train_and_evaluate(train_index, val_index)
    cv_scores.append(accuracy)

# Print cross-validation results
print("PyTorch Cross-validation scores:", cv_scores)
print("PyTorch Mean CV accuracy:", sum(cv_scores) / len(cv_scores))




The above results show the variability of the scores, indicating that the performance of the model is not consistent on different data splits, especially the third fold, which exhibits a very low accuracy. The average accuracy is 0.6971, which is not a good performance.

13.Select the best model from previous and apply to test data.From previous items the best moder is rf_model2

In [ ]:
#Load test data
test_data = pd.read_csv('/content/drive/MyDrive/Machine Learning/IS41070-MACHINE LEARNING FOUNDATIONS/test data.csv')
test_data = test_data.drop(columns=['authors','link','date'])

In [ ]:
test_data.head()

In [ ]:
#Preprocess the test data
test_data['short_description'].fillna('',inplace=True)
test_data['short_description_pre']=test_data['short_description'].apply(clean_text)

In [ ]:
train_data.head()

In [ ]:
train_data['short_description'].fillna('',inplace=True)
train_data['short_description_pre'] = train_data['short_description'].apply(clean_text)

In [ ]:
x_train2 = train_data['short_description_pre'].astype(str)
y_train2 = train_data['category']
x_test2 = test_data['short_description_pre'].astype(str)
y_test2 = test_data['category']
vectorizer = TfidfVectorizer(max_features=5000)
x_train_tf2 = vectorizer.fit_transform(x_train2)
x_test_tf2 = vectorizer.transform(x_test2)

#apply test data to rf_model2
rf_model2 = RandomForestClassifier()
rf_model2.fit(x_train_tf2,y_train2)
y_pred_test2 = rf_model2.predict(x_test_tf2)
print(classification_report(y_test2,y_pred_test2))


The above results show the results of applying rf_model2 with test data. It is found that the model accuracy does not improve but slightly decreases with test data. In addition, the classification of the WELLNESS category was improved using test data.

14.Choose the model with best performance,retrain it and apply to test_data.From 11 we know rf_model3 is the model with best performance.

In [ ]:
#Load test data
test_data = pd.read_csv('/content/drive/MyDrive/Machine Learning/IS41070-MACHINE LEARNING FOUNDATIONS/test data.csv')

In [ ]:
test_data.head()

In [ ]:
#Preprocess the test data
test_data['headline'].fillna('',inplace=True)
test_data['headline_pre']=test_data['headline'].apply(clean_text)

In [ ]:
combined_data.info()

In [ ]:
#Preprocess
combined_data['headline_pre'] = combined_data['headline'].apply(clean_text)

In [ ]:
combined_data.head()

In [ ]:
combined_data.info()

In [ ]:
#Retrain the model with more data, that is combined_data
x_train_combined= combined_data['headline_pre'].astype(str)
y_train_combined= combined_data['category']
x_test= test_data['headline_pre'].astype(str)
y_test= test_data['category']
vectorizer = TfidfVectorizer(max_features=5000)
x_train_tf_combined = vectorizer.fit_transform(x_train_combined)
x_test_tf1= vectorizer.transform(x_test)
#Retrain the rf_model3 using combined data
rf_model4 = RandomForestClassifier()
rf_model4.fit(x_train_tf_combined,y_train_combined)

In [ ]:
y_pred_test_rf = rf_model4.predict(x_test_tf1)
print(classification_report(y_test,y_pred_test_rf))


Compared with rf_model3 in 11,rf_model4 uses more data to train the model, but it does not improve. On the contrary, some indicators decline (accuracy from 0.90 to 0.89), which may mean that the current amount of data is large enough. Further increasing the amount of data did not bring significant performance gains.

References

How to Use k-fold Cross Validation with DataLoaders in PyTorch | Saturn Cloud Blog. (2024, January 14). https://saturncloud.io/blog/how-to-use-kfold-cross-validation-with-dataloaders-in-pytorch/

Kampakis, S. (2023, December 9). What is the F-1 measure and why is it useful for imbalanced class problems? The Data Scientist. https://thedatascientist.com/f-1-measure-useful-imbalanced-class-problems/
